### GUI 꾸미기

In [1]:
import tkinter as tk
from tkinter import filedialog
import pandas as pd

# 마스터 데이터 업데이트하기
def update_master (df, idx, var):
    df['Item'][idx] = var
    df.to_csv('Master/MailMaster.csv', index = False)

# 경로 변경하기
def change_Path (df, idx, fileyn):
    # 파일 경로, 폴더 경로 구분하기
    if fileyn[idx] == 0 :
        f_selected = filedialog.askdirectory(title = lbPath[idx].cget('text'))
        f_selected = f_selected.replace('/','\\')
    else :
        f_selected = filedialog.askopenfile(title = lbPath[idx].cget('text'))
        f_selected = f_selected.name.replace('/','\\')
    # 텍스트 박스에 정보 업데이트하고 마스터 데이터 업데이트
    txtPath[idx].delete('1.0', tk.END)
    txtPath[idx].insert(tk.END, chars=f_selected)
    update_master (df, idx, f_selected)    

# Master Data 가져오기
try : 
    df = pd.read_csv('Master/MailMaster.csv')

except :
    d = {'Item' : ['폴더 경로를 설정해 주세요'] * 5}
    df = pd.DataFrame(data=d)

# GUI 창 생성하고 기본 정보 설정
win = tk.Tk()
win.geometry("680x500")
win.title('개별 급여 명세표 생성')

# 폴더/파일 경로 설정 frame 생성
frame1 = tk.Frame(win, padx = 5, pady=5, width = 100)
frame1.pack(padx = 10, pady=5)

# 개체 초기화
lbPath = []
txtPath = []
btnPath = []

# 각 행의 라벨 정보 List로 정리
lbName = ['원본 파일 위치','결과 저장 폴더','결과 저장 폴더(pw)','개인 메일 Master']
# 각 행에서 다루는 값이 폴더일때는 0, 파일일때는 1로 구분자
fileyn = [1, 0, 0, 1]

# 파일 / 폴더 경로 가져오기 GUI 구성
for idx, x in enumerate(lbName):
    # 라벨, 텍스트 상자, 버튼 쌓기
    lbPath.append(tk.Label(frame1, text=x, width = 15, padx = 5, pady=5, anchor = 'w'))
    txtPath.append(tk.Text(frame1, width = 50, height = 1, padx = 5, pady=5, 
                            background='lightgrey'))
    btnPath.append(tk.Button(frame1,text="Path 변경", width = 10,padx = 5, pady=5,
                        command = lambda i = idx : change_Path(df,i,fileyn)))
    # 텍스트 상자 초깃값 넣기
    txtPath[idx].insert(tk.INSERT, chars=df['Item'][idx])

    # grid 배치하기
    lbPath[idx].grid(row=idx, column=0, padx = 5, pady=5)
    txtPath[idx].grid(row=idx, column=1, padx = 5, pady=5)
    btnPath[idx].grid(row=idx, column=2, padx = 5, pady=5)

# 폴더/파일 경로 설정 frame 생성
frame2 = tk.Frame(win, padx = 5, pady=5, width = 100)
frame2.pack(padx = 10, pady=5)

lbMtitle = tk.Label(frame2, text = '메일 제목', width = 80, padx = 5, pady = 5, anchor = 'w')
txtMTitle = tk.Text(frame2, width = 80, height = 1, padx = 5, pady = 5)
lbMbody = tk.Label(frame2, text = '메일 본문', width = 80, padx = 5, pady = 5, anchor = 'w')
txtMbody = tk.Text(frame2, width = 80, height = 8, padx = 5, pady = 5)

lbMtitle.pack()
txtMTitle.pack()
lbMbody.pack()
txtMbody.pack()

# 실행 버튼 생성하기
frame3 = tk.Frame(win, padx = 5, pady=5, width = 100)
frame3.pack(padx = 10, pady=5)

var = tk.IntVar()
chkMail = tk.Checkbutton(frame3, text = '메일 발송 여부', variable= var , width = 30)
btnPay = tk.Button(frame3, text = '개별 급여 명세표 만들기', width = 30, padx=5, pady=5)

# 버튼 배치하기
chkMail.grid(row=0, column=0, padx =5, pady =5)
btnPay.grid(row=0, column=1, padx =5, pady =5)

win.mainloop()


### GUI와 개별 급여 명세표 자동 생성 전송 기능 연결하기

In [1]:
import tkinter as tk
from tkinter import filedialog
import pandas as pd
import win32com.client
from PyPDF2 import PdfReader, PdfWriter
import os
import smtplib
from os.path import basename
from email.mime.application import MIMEApplication
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText

# 개별 급여 명세표 자동 생성기 TPath : 원본 파일 위치, RPath : 결과 저장 폴더, 
#       pwPath : 암호 걸린 결과 파일 저장 폴더, mPath : 개인 메일 정보 파일,
#       mTitle : 메일 제목, mBody : 메일 본문, chkM : 메일 발송여부(1이면 발송)
def AutoPay(TPath, RPath, pwPath, mPath, mTitle, mBody, chkM):
    # 개인 메일 정보 불러오기
    dfmail = pd.read_csv(mPath)
    id = dfmail['id'][0]
    pw = dfmail['pw'][0]

    # 메일 서비스를 위한 준비
    # 세션 생성
    if chkM == 1 :
        s = smtplib.SMTP('smtp.gmail.com', 587)

        # TLS 보안 시작
        s.starttls()

        # 로그인 인증
        s.login(id,pw)

    # 파일 경로 지정하기 TPath : 취합급여명세표 파일,
    #                  RPath : PDF 결과 저장 폴더, 
    #                  pwPath : 암호 설정된 PDF 저장 폴더

    # 각각의 시트의 값 읽어오기 df : Raw, dfM : Master, dfP : Private
    df = pd.read_excel(TPath, sheet_name = 'Raw')
    dfM = pd.read_excel(TPath, sheet_name = 'Master')
    dfP = pd.read_excel(TPath, sheet_name = 'Private')

    # 사번 정보 기준으로 df와 dfP 데이터 결합하기
    df = df.merge(dfP, on='사번')

    # None 값 0으로 대체하기
    df = df.fillna(0)

    #엑셀 실행하고 Format 시트를 ws로 선언하기
    excel = win32com.client.Dispatch("Excel.Application")
    excel.Visible = True
    wb = excel.Workbooks.Open(TPath)
    ws = wb.Sheets("Format")

    #각 인원에 대한 정보 반복문
    for index, row in df.iterrows():
        #각 인원에 대한 사번, 이름 정보 가져오기
        ws.Cells(2,2).Value = row['사번']
        ws.Cells(2,4).Value = row['이름_x']

        #지급 공제 입력줄 초기화
        pnum = 5
        cnum = 5

        # 칼럼 명에 대한 반복
        for cName in df:
            # 구분자 생성    
            Sep = dfM['구분'].where(dfM['항목'] == cName).dropna()
            # 구분자가 값이 없을때
            if Sep.empty:
                if cName == '비고':
                    ws.Cells(26,1).Value = row[cName]
            # 구분이 지급일 경우
            elif Sep.iloc[0] =="지급" and row[cName] > 0 :
                ws.Cells(pnum,1).Value = cName
                ws.Cells(pnum,2).Value = row[cName]
                pnum += 1
            # 구분이 공제일 경우
            elif Sep.iloc[0] =="공제" and row[cName] > 0 :
                ws.Cells(cnum,3).Value = cName
                ws.Cells(cnum,4).Value = row[cName]
                cnum += 1

        # PDF 파일로 파일 저장
        FName = str(row['사번']) + str(row['이름_x']) + '.pdf'
        ws.ExportAsFixedFormat(0, os.path.join(RPath,FName) )

        # 지급/공제 값 초기화     
        ws.Range("A5:D20").Value = ''

        # 암호가 걸린 위해 PDF 파일명 불러오기
        input_pdf = PdfReader(open(os.path.join(RPath,FName), "rb"))

        # PDF 파일 저장을 위한 Writer 생성하고 패스워드 설정하여 내용 입력
        output_pdf = PdfWriter()
        output_pdf.append_pages_from_reader(input_pdf)
        output_pdf.encrypt(str(row['PW']))

        # 만들어진 결과 파일을 저장 ※ wb 옵션은 파일을 저장하기 위한 binary 포멧이라는의미
        output_pdf.write(open(os.path.join(pwPath, 'pw_' + FName), "wb"))   

        # 메일 보내기
        if chkM == 1 :
            # 보낼 메시지 설정
            msg = MIMEMultipart()
            msg['Subject'] = mTitle
            msg['From'] = id
            msg['To'] = row['e-mail']
            msg.attach(MIMEText(mBody))

            # 첨부 파일을 보내기 위한 사전 작업
            file = os.path.join(pwPath, 'pw_' + FName)
            fil = open(file, "rb")

            # 첨부 파일을 메일에 붙이는 작업
            part = MIMEApplication(fil.read(),Name=basename(file))

            # 첨부된 파일에 대한 정보 추가
            part['Content-Disposition'] = 'attachment; filename="%s"' % basename(file)
            msg.attach(part)


            # 메일 보내기
            s.sendmail(id, row['e-mail'], msg.as_string())

    # 세션 종료
    if chkM == 1 :
        s.quit()

    # 엑셀 닫고 종료
    wb.Close(False)
    excel.Quit()

# 마스터 데이터 업데이트하기
def update_master (df, idx, var):
    df['Item'][idx] = var
    df.to_csv('Master/MailMaster.csv', index = False)

# 경로 변경하기
def change_Path (df, idx, fileyn):
    # 파일 경로, 폴더 경로 구분하기
    if fileyn[idx] == 0 :
        f_selected = filedialog.askdirectory(title = lbPath[idx].cget('text'))
        f_selected = f_selected.replace('/','\\')
    else :
        f_selected = filedialog.askopenfile(title = lbPath[idx].cget('text'))
        f_selected = f_selected.name.replace('/','\\')
    # 텍스트 박스에 정보 업데이트하고 마스터 데이터 업데이트
    txtPath[idx].delete('1.0', tk.END)
    txtPath[idx].insert(tk.END, chars=f_selected)
    update_master (df, idx, f_selected)    

# Master Data 가져오기
try : 
    df = pd.read_csv('Master/MailMaster.csv')

except :
    d = {'Item' : ['폴더 경로를 설정해 주세요'] * 5}
    df = pd.DataFrame(data=d)

# GUI 창 생성하고 기본 정보 설정
win = tk.Tk()
win.geometry("680x500")
win.title('개별 급여 명세표 생성')

# 폴더/파일 경로 설정 frame 생성
frame1 = tk.Frame(win, padx = 5, pady=5, width = 100)
frame1.pack(padx = 10, pady=5)

# 개체 초기화
lbPath = []
txtPath = []
btnPath = []

# 각 행의 라벨 정보 List로 정리
lbName = ['원본 파일 위치','결과 저장 폴더','결과 저장 폴더(pw)','개인 메일 Master']
# 각 행에서 다루는 값이 폴더일때는 0, 파일일때는 1로 구분자
fileyn = [1, 0, 0, 1]

# 파일 / 폴더 경로 가져오기 GUI 구성
for idx, x in enumerate(lbName):
    # 라벨, 텍스트 상자, 버튼 쌓기
    lbPath.append(tk.Label(frame1, text=x, width = 15, padx = 5, pady=5, anchor = 'w'))
    txtPath.append(tk.Text(frame1, width = 50, height = 1, padx = 5, pady=5, 
                            background='lightgrey'))
    btnPath.append(tk.Button(frame1,text="Path 변경", width = 10,padx = 5, pady=5,
                        command = lambda i = idx : change_Path(df,i,fileyn)))
    # 텍스트 상자 초깃값 넣기
    txtPath[idx].insert(tk.INSERT, chars=df['Item'][idx])

    # grid 배치하기
    lbPath[idx].grid(row=idx, column=0, padx = 5, pady=5)
    txtPath[idx].grid(row=idx, column=1, padx = 5, pady=5)
    btnPath[idx].grid(row=idx, column=2, padx = 5, pady=5)

# 폴더/파일 경로 설정 frame 생성
frame2 = tk.Frame(win, padx = 5, pady=5, width = 100)
frame2.pack(padx = 10, pady=5)

lbMtitle = tk.Label(frame2, text = '메일 제목', width = 80, padx = 5, pady = 5, anchor = 'w')
txtMTitle = tk.Text(frame2, width = 80, height = 1, padx = 5, pady = 5)
lbMbody = tk.Label(frame2, text = '메일 본문', width = 80, padx = 5, pady = 5, anchor = 'w')
txtMbody = tk.Text(frame2, width = 80, height = 8, padx = 5, pady = 5)

lbMtitle.pack()
txtMTitle.pack()
lbMbody.pack()
txtMbody.pack()

# 실행 버튼 생성하기
frame3 = tk.Frame(win, padx = 5, pady=5, width = 100)
frame3.pack(padx = 10, pady=5)

var = tk.IntVar()
chkMail = tk.Checkbutton(frame3, text = '메일 발송 여부', variable= var , width = 30)
btnPay = tk.Button(frame3, text = '개별 급여 명세표 만들기', width = 30, padx=5, pady=5,
                    command= lambda : AutoPay(df['Item'][0],df['Item'][1],df['Item'][2], df['Item'][3],
                        txtMTitle.get("1.0",tk.END), txtMbody.get("1.0",tk.END), var.get()))

# 버튼 배치하기
chkMail.grid(row=0, column=0, padx =5, pady =5)
btnPay.grid(row=0, column=1, padx =5, pady =5)

win.mainloop()
